In [1]:
import os, os.path, time
import PIL
import glob
import cv2
import numpy as np
from PIL import Image

In [2]:


#number = 0
def format_image(image):
    #global number
    #number += 1
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image = clahe.apply(image)
    else:
        image = cv2.imdecode(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
    faces = cv2.CascadeClassifier('haarcascade_frontalface_default.xml').detectMultiScale(
        image,
        scaleFactor=1.3,
        minNeighbors=5
    )
    
    #print("number of faces in image(",number,") : ",len(faces))
    if not len(faces) > 0:
        return None
    max_area_face = faces[0]
    for face in faces:
        if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
            max_area_face = face
    # Chop image to face
    face = max_area_face
    image = image[face[1]:(face[1] + face[2]), face[0]:(face[0] + face[3])]
    # Resize image to network size
    try:
        image = cv2.resize(image, (48, 48), interpolation=cv2.INTER_CUBIC)
        return image
    except Exception:
        print("[+] Problem during resize")
        return None

#filename = "../RafD/front/Rafd090_01_Caucasian_female_angry_frontal.jpg"
#frame = cv2.imread(filename)
#resultImage = format_image(frame)
#print("pixel values",resultImage)

In [3]:
emotions = {'angry' : 0, 'disgusted' : 1, 'fearful': 2, 'happy' : 3, 'sad' : 4 , 'surprised' : 5, 'neutral' : 6}
image_train = []
labels_train = []


In [ ]:
# 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

print("Start")

for filename in glob.glob('../RafD/front/*.jpg'):
    #filename = "../RafD/front/Rafd090_01_Caucasian_female_angry_frontal.jpg"
    frame = cv2.imread(filename)
    resultImage = format_image(frame)
    params = str(filename).split("_")    
    if emotions.get(params[4]) is not None:
        labels_train.append(emotions.get(params[4]))
        image_train.append(resultImage.reshape(1,2304))
#np.savetxt("RafD/labels_train.csv", labels_train,fmt='%i', delimiter=",")
np.save("RafD/labels_train", labels_train)
np.save("RafD/images_train",image_train)
print("end of program")

Start


print("Start")
im = Image.open('../RafD/front/Rafd090_01_Caucasian_female_angry_frontal.jpg')
filename = "Rafd090_01_Caucasian_female_angry_frontal"
params = str(filename).split("_")   


print(im.getpixel((0,0)))

if emotions.get(params[4]) is not None:
    print(emotions.get(params[4]))
    x = 0
    y = 0
    image_pixels = []
    for x in range(48):
        for y in range(48):
            image_pixels.append(im.getpixel((x,y)))
    image_train.append(image_pixels)
            
print(len(image_pixels),image_train,"end of program")

#convertitn images to 48*48 and saving back to disk
size = 48,48
for filename in glob.glob('../RafD/front/*.jpg'):
    im = Image.open(filename)
    im.resize(size, PIL.Image.ANTIALIAS)
    im = im.convert("L")
    im.save(filename)
    
    
for file in os.listdir("../RafD/front"):
    if file.endswith(".jpg") or file.endswith(".png"):
        params = str(file).split("_")
